In [23]:
import numpy as np
import pandas as pd

# Load weather data (ensure the file path is correct)
weather_data = pd.read_csv(r'C:\Users\Yakub\PycharmProjects\HRES_OPtimization\weather_data.csv')

# Display the first few rows of the weather data to ensure it is loaded correctly
print("Weather data loaded:")
print(weather_data.head())

# Techno-economic parameters
economic_params = {
    'pv_cost': 650,  # USD/kW
    'battery_cost': 174,  # USD/kWh
    'wind_cost': 1300,  # USD/kW
    'discount_rate': 0.08,
    'project_lifetime': 20,
    'battery_lifetime': 5,
    'battery_O&M': 5,  # USD/kWh/year
    'pv_efficiency': 0.85,
    'pv_temp_coeff': -0.004,  # %/°C
    'inv_efficiency': 0.95,
    'interest_rate': 0.03,
    'inflation_rate': 0.02
}

# Specifications for components
specs = {
    'pv_rated_power': 200,  # W
    'pv_temp_coeff': 0.004,  # %/°C
    'pv_standard_radiation': 1,  # kW/m2
    'battery_self_discharge': 0.01,
    'wind_rated_power': 3.3,  # kW
    'wind_cut_in_speed': 2.7,  # m/s
    'wind_cut_out_speed': 25.0,  # m/s
    'wind_rated_speed': 12.5,  # m/s
    'load_demand': 1000  # kW
}



Weather data loaded:
   Datetime  temperature  wind_speed  wind_direction  humidity  pressure  \
0  1/1/2018          1.3         3.1             290      36.1       2.4   
1  1/2/2018          1.5         2.3             290      50.6       3.4   
2  1/3/2018          0.3         2.9             320      30.0       1.8   
3  1/4/2018          0.6         1.7             290      38.5       2.4   
4  1/5/2018          2.1         2.5             290      36.5       2.5   

   solar_radiation  
0             1.88  
1             1.65  
2             1.77  
3             1.77  
4             1.90  


In [24]:
# Function to calculate NPC
def calculate_npc(wind_capacity, pv_capacity, battery_capacity, weather_data, economic_params, specs):
    # Print the capacities being evaluated
    print(f"Evaluating NPC for wind: {wind_capacity} kW, PV: {pv_capacity} kW, battery: {battery_capacity} kWh")

    # Calculate energy production from wind and PV based on weather data
    wind_energy = np.where(
        weather_data['wind_speed'] < specs['wind_cut_in_speed'], 0,
        np.where(
            weather_data['wind_speed'] < specs['wind_cut_out_speed'],
            specs['wind_rated_power'] * ((weather_data['wind_speed'] ** 3 - specs['wind_cut_in_speed'] ** 3) /
                                         (specs['wind_rated_speed'] ** 3 - specs['wind_cut_in_speed'] ** 3)),
            specs['wind_rated_power']
        )
    ) * wind_capacity

    pv_energy = pv_capacity * economic_params['pv_efficiency'] * (
            weather_data['solar_radiation'] / specs['pv_standard_radiation']) * (
                        1 + economic_params['pv_temp_coeff'] * (weather_data['temperature'] - 25))

    # Simple battery model to store excess energy
    battery_energy = np.zeros(len(weather_data))
    for t in range(1, len(weather_data)):
        excess_energy = max(0, wind_energy[t] + pv_energy[t] - specs['load_demand'])
        battery_energy[t] = min(battery_capacity,
                                battery_energy[t - 1] * (1 - specs['battery_self_discharge']) + excess_energy)

    # Calculate NPC
    wind_npc = economic_params['wind_cost'] * wind_capacity
    pv_npc = economic_params['pv_cost'] * pv_capacity
    battery_npc = economic_params['battery_cost'] * battery_capacity
    total_npc = wind_npc + pv_npc + battery_npc

    # Print the calculated NPC for debugging
    print(f"Calculated NPC: {total_npc}")
    return total_npc

In [25]:
# TLBO algorithm
def tlbo_optimization(weather_data, economic_params, specs, num_iterations=100, population_size=30):
    # Initialize the population with meaningful values
    np.random.seed(42)  # For reproducibility
    population = np.random.uniform(low=[10, 10, 10], high=[100, 100, 100], size=(population_size, 3))

    # Print initial population for debugging
    print("Initial population:")
    print(population)

    best_solution = None
    best_npc = float('inf')

    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}/{num_iterations}")
        # Teacher Phase
        mean_solution = np.mean(population, axis=0)
        fitness_values = [calculate_npc(ind[0], ind[1], ind[2], weather_data, economic_params, specs) for ind in population]
        teacher = population[np.argmin(fitness_values)]

        for i in range(population_size):
            new_solution = population[i] + np.random.rand() * (teacher - np.random.rand() * mean_solution)
            new_solution = np.clip(new_solution, [10, 10, 10], [100, 100, 100])  # Ensure capacities are within bounds
            if calculate_npc(new_solution[0], new_solution[1], new_solution[2], weather_data, economic_params, specs) < calculate_npc(population[i][0], population[i][1], population[i][2], weather_data, economic_params, specs):
                population[i] = new_solution

        # Learner Phase
        for i in range(population_size):
            j = np.random.randint(population_size)
            if i != j:
                if calculate_npc(population[i][0], population[i][1], population[i][2], weather_data, economic_params, specs) < calculate_npc(population[j][0], population[j][1], population[j][2], weather_data, economic_params, specs):
                    new_solution = population[i] + np.random.rand() * (population[i] - population[j])
                else:
                    new_solution = population[i] + np.random.rand() * (population[j] - population[i])
                new_solution = np.clip(new_solution, [10, 10, 10], [100, 100, 100])  # Ensure capacities are within bounds
                if calculate_npc(new_solution[0], new_solution[1], new_solution[2], weather_data, economic_params, specs) < calculate_npc(population[i][0], population[i][1], population[i][2], weather_data, economic_params, specs):
                    population[i] = new_solution

        # Update the best solution
        for ind in population:
            npc = calculate_npc(ind[0], ind[1], ind[2], weather_data, economic_params, specs)
            if npc < best_npc:
                best_npc = npc
                best_solution = ind

    return best_solution, best_npc

In [26]:
# Run the optimization
best_solution, best_npc = tlbo_optimization(weather_data, economic_params, specs)
print(f"Best Solution: Wind Capacity: {best_solution[0]} kW, PV Capacity: {best_solution[1]} kW, Battery Capacity: {best_solution[2]} kWh")
print(f"Net Present Cost: {best_npc}")

Initial population:
[[43.7086107  95.56428758 75.87945476]
 [63.87926358 24.04167764 24.03950683]
 [15.2275251  87.95585312 64.10035106]
 [73.726532   11.85260449 97.29188669]
 [84.91983767 29.11051996 26.36424705]
 [26.50640589 37.38180187 57.22807885]
 [48.87505168 36.21062262 65.06676053]
 [22.55444746 36.29301837 42.9725659 ]
 [51.04629858 80.66583653 27.97064039]
 [56.28109946 63.3173112  14.18053714]
 [64.67903667 25.34717113 15.85464337]
 [95.39969835 96.90688298 82.75576133]
 [37.41523923 18.79049026 71.58097239]
 [49.61372444 20.98344114 54.56592191]
 [13.0949669  91.83883619 33.29019834]
 [69.62700559 38.05399685 56.80612191]
 [59.20392514 26.636901   97.2626165 ]
 [79.7619541  94.55490474 90.53446154]
 [63.81099809 92.96868115 17.96432518]
 [27.63845762 14.070456   39.27972977]
 [44.98095607 34.42141286 84.58637582]
 [42.1077994  35.28410587 58.84264748]
 [22.68318025 82.19772827 16.70955793]
 [98.81982429 79.50202924 27.88441134]
 [10.49699054 83.39152856 73.61716095]
 [75.

Calculated NPC: 122436.70066118878
Evaluating NPC for wind: 59.203925140895166 kW, PV: 26.636900997297435 kW, battery: 97.26261649881027 kWh
Calculated NPC: 111202.78360220004
Evaluating NPC for wind: 81.01431832340022 kW, PV: 92.55016884468708 kW, battery: 94.40958367407215 kWh
Calculated NPC: 181903.49112875544
Evaluating NPC for wind: 79.76195410250031 kW, PV: 94.55490474077702 kW, battery: 90.5344615384884 kWh
Calculated NPC: 180904.22472245243
Evaluating NPC for wind: 49.915003115499495 kW, PV: 66.7843160510068 kW, battery: 13.36904871235551 kWh
Calculated NPC: 110625.52395925362
Evaluating NPC for wind: 63.810998092997664 kW, PV: 92.96868115208052 kW, battery: 17.964325184672756 kWh
Calculated NPC: 146509.73285188235
Evaluating NPC for wind: 27.64650795483945 kW, PV: 13.978180856177921 kW, battery: 39.36759211555518 kWh
Calculated NPC: 51876.23892591354
Evaluating NPC for wind: 27.638457617723066 kW, PV: 14.070456001948425 kW, battery: 39.27972976869379 kWh
Calculated NPC: 51910.

Calculated NPC: 105790.14265316701
Evaluating NPC for wind: 56.28109945722505 kW, PV: 63.31731119758382 kW, battery: 14.180537144799796 kWh
Calculated NPC: 116789.0950360172
Evaluating NPC for wind: 64.67903667112945 kW, PV: 25.347171131856236 kW, battery: 15.854643368675156 kWh
Calculated NPC: 103317.11685432431
Evaluating NPC for wind: 54.430340927913804 kW, PV: 10.679578453250915 kW, battery: 50.78094091669429 kWh
Calculated NPC: 86537.05292040585
Evaluating NPC for wind: 56.11827554028662 kW, PV: 13.095294363130776 kW, battery: 45.028666872227824 kWh
Calculated NPC: 89300.68757417524
Evaluating NPC for wind: 64.67903667112945 kW, PV: 25.347171131856236 kW, battery: 15.854643368675156 kWh
Calculated NPC: 103317.11685432431
Evaluating NPC for wind: 82.14239187458404 kW, PV: 69.76612025778509 kW, battery: 80.0844156594651 kWh
Calculated NPC: 166067.7759292665
Evaluating NPC for wind: 66.09683141448022 kW, PV: 39.78082223673843 kW, battery: 15.720251525742128 kWh
Calculated NPC: 114518

Calculated NPC: 152812.6931880792
Evaluating NPC for wind: 42.261915568984534 kW, PV: 20.428215357261674 kW, battery: 87.67930832880342 kWh
Calculated NPC: 83475.02987111178
Evaluating NPC for wind: 62.767121879503954 kW, PV: 11.885777375173648 kW, battery: 31.64922557864846 kWh
Calculated NPC: 94829.97898790284
Evaluating NPC for wind: 21.833527917830025 kW, PV: 28.938650801869485 kW, battery: 100.0 kWh
Calculated NPC: 64593.7093143942
Evaluating NPC for wind: 42.261915568984534 kW, PV: 20.428215357261674 kW, battery: 87.67930832880342 kWh
Calculated NPC: 83475.02987111178
Evaluating NPC for wind: 66.09683141448022 kW, PV: 39.78082223673843 kW, battery: 15.720251525742128 kWh
Calculated NPC: 114518.7390581834
Evaluating NPC for wind: 67.38017242196918 kW, PV: 89.84914683186939 kW, battery: 52.49934326457544 kWh
Calculated NPC: 155131.05531731114
Evaluating NPC for wind: 64.94539763973343 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 92669.01693165346
Evaluating NPC for wind: 66.0

Calculated NPC: 45471.236440067376
Evaluating NPC for wind: 48.779520983564694 kW, PV: 27.147791441977176 kW, battery: 71.09457237569586 kWh
Calculated NPC: 93429.89730929035
Evaluating NPC for wind: 35.39496974303705 kW, PV: 42.449479846792975 kW, battery: 61.07079854537547 kWh
Calculated NPC: 84231.94151325893
Evaluating NPC for wind: 10.0 kW, PV: 76.87399059816896 kW, battery: 10.0 kWh
Calculated NPC: 64708.093888809824
Evaluating NPC for wind: 59.79580006562079 kW, PV: 46.64104194597964 kW, battery: 41.99835717679376 kWh
Calculated NPC: 115358.9314989559
Evaluating NPC for wind: 10.0 kW, PV: 59.87434585706715 kW, battery: 100.0 kWh
Calculated NPC: 69318.32480709365
Evaluating NPC for wind: 48.831114202885686 kW, PV: 42.64899053454073 kW, battery: 27.338713958135727 kWh
Calculated NPC: 95959.22853991848
Evaluating NPC for wind: 21.833527917830025 kW, PV: 28.938650801869485 kW, battery: 100.0 kWh
Calculated NPC: 64593.7093143942
Evaluating NPC for wind: 64.94539763973343 kW, PV: 10.0

...

Calculated NPC: 29522.95648333784
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 13.534177959570167 kW, PV: 13.534177959570167 kW, battery: 13.534177959570167 kWh
Calculated NPC: 28746.59398612703
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.345519286998249 kW, PV: 10.345519286998249 kW, battery: 10.345519286998249 kWh
Calculated NPC: 21973.882965584282
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 13.780890158115398 kW, PV: 13.780890158115398 kW, battery: 13.780890158115398 kWh
Calculated NPC: 29270.61069583711
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 18.81517403932704 kW, PV: 18.81517403932704 kW, battery: 18.81517403932704 kWh
Calculated NPC: 39963.42965953063
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, b

Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240

Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.000000000000002 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.000000000000002 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10

Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240

Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240.0
Evaluating NPC for wind: 10.0 kW, PV: 10.0 kW, battery: 10.0 kWh
Calculated NPC: 21240